# Tensorflow手工搭建CNN回顾

在利用Tensorflow手工搭建CNN的时候，我们主要经历这么几个步骤：

    1、定义训练样本和label，这里仍然是采用批处理的方式。
    
    2、采用两层卷积层和池化层。
    
    3、最后跟一层全连接层，并在最后一层加入dropout
    
    4、激活函数使用Relu，具体原因可以理解为防止梯度消失等问题吧。
    
    5、loss选择的还是log损失，这和问题是一个分类问题有关。
    
    6、优化没有选在传统的gradient-descent了，而是选择使用AdaDelta，基本原理估计还是GD，猜想是对学习率等内容做了调整。

呃，现在问题是，我们发现这样如果要搭一个很多层的网络的话，一层一层去配太麻烦，而且万一漏了，要一层层检查就很尴尬。

因此，需要多tensorflow做一个封装，不需要自己去搭建，告诉代码我要几层，他们怎么样就好了。

keras就是实现这个封装的包，它的底层是用tensorflow来实现的。

那么，更新之后的神经网络格式是这样的：

    1、keras实现cnn
    
    2、2层网络
    
    3、卷积（&池化）之后，加入规范层
    
    4、使用RELU激活函数。

# 数据预处理

## 加载相关的包

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D
from keras.layers import MaxPool2D,Flatten,Dropout,ZeroPadding2D,BatchNormalization
from keras.utils import np_utils
from keras.models import save_model,load_model
from keras.models import Model

## 加载数据

In [2]:
df=pd.read_csv('input/train.csv')

## 乱序&分割

将原数据集进行乱序，并把特征和labels进行分割。

**乱序**

In [3]:
data=df.as_matrix()

df=None

# 乱序
np.random.shuffle(data)

**分割特征并改变格式**

In [6]:
x_train=data[:,1:]
# 把1*784的数据转化成28*28的格式
x_train=x_train.reshape(data.shape[0],28,28,1).astype('float32')
x_train=x_train/255.0

x_train.shape

(42000, 28, 28, 1)

**分割labels并one-hot**

In [7]:
# 分割得到labels，并利用numpy进行one-hot
y_train=np_utils.to_categorical(data[:,0],10).astype('float32')
y_train.shape

(42000, 10)

## 设置神经网络参数

这里设置三个参数：

    1、batch大小：64
    
    2、卷积滤镜个数：32
    
    3、池化核大小：2*2

In [8]:
# batch-size
batch_size=64

# 卷积滤镜
n_filters=32

# 池化核
pool_size=(2,2)

## 定义模型

这是最核心的地方，利用keras我们能够很方便的定义一个3层的模型

**第一层:卷积池化**

In [9]:
cnn_net=Sequential()

# 第一层
cnn_net.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),input_shape=(28,28,1)))
cnn_net.add(Activation('relu'))
cnn_net.add(BatchNormalization(epsilon=1e-6,axis=1))
cnn_net.add(MaxPool2D(pool_size=pool_size))

**第二层：卷积池化**

In [10]:
cnn_net.add(ZeroPadding2D((1,1)))
cnn_net.add(Conv2D(48,kernel_size=(3,3)))
cnn_net.add(Activation('relu'))
cnn_net.add(BatchNormalization(epsilon=1e-6,axis=1))
cnn_net.add(MaxPool2D(pool_size=pool_size))

**第三层：卷积池化**

In [11]:
cnn_net.add(ZeroPadding2D((1,1)))
cnn_net.add(Conv2D(64,kernel_size=(2,2)))
cnn_net.add(Activation('relu'))
cnn_net.add(BatchNormalization(epsilon=1e-6,axis=1))
cnn_net.add(MaxPool2D(pool_size=pool_size))

**第四层：全连接**

In [12]:
cnn_net.add(Dropout(0.25))
cnn_net.add(Flatten())

cnn_net.add(Dense(3168))
cnn_net.add(Activation('relu'))

cnn_net.add(Dense(10))
cnn_net.add(Activation('softmax'))

## 查看模型结构

由于绘图的功能一直搞不定，勉强用这个看一下把

In [13]:
cnn_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        104       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 48)        13872     
_________________________________________________________________
activation_2 (Activation)    (None, 13, 13, 48)        0         
__________

## 训练模型

In [31]:
# 编译模型
cnn_net.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# 运行/训练模型
cnn_net.fit(x_train,y_train,batch_size=batch_size,epochs=50,verbose=1,validation_split=0.2)

# 保存模型
cnn_net.save('save_model/cnn_net_model.h5')

Train on 33600 samples, validate on 8400 samples
Epoch 1/50
33600/33600 [==============================] - 80s 2ms/step - loss: 0.2255 - acc: 0.9332 - val_loss: 0.0701 - val_acc: 0.9801
Epoch 2/50
33600/33600 [==============================] - 80s 2ms/step - loss: 0.0731 - acc: 0.9774 - val_loss: 0.0685 - val_acc: 0.9781
Epoch 3/50
33600/33600 [==============================] - 77s 2ms/step - loss: 0.0533 - acc: 0.9838 - val_loss: 0.0638 - val_acc: 0.9832
Epoch 4/50
33600/33600 [==============================] - 82s 2ms/step - loss: 0.0424 - acc: 0.9869 - val_loss: 0.0443 - val_acc: 0.9877
Epoch 5/50
33600/33600 [==============================] - 83s 2ms/step - loss: 0.0389 - acc: 0.9881 - val_loss: 0.0377 - val_acc: 0.9886
Epoch 6/50
33600/33600 [==============================] - 80s 2ms/step - loss: 0.0300 - acc: 0.9906 - val_loss: 0.0477 - val_acc: 0.9865
Epoch 7/50
33600/33600 [==============================] - 80s 2ms/step - loss: 0.0314 - acc: 0.9905 - val_loss: 0.0461 - val_acc:

## 使用保存好的模型进行预测

In [37]:
df=pd.read_csv('input/test.csv')

x_valid=df.values.astype('float32')

n_valid=x_valid.shape[0]

x_valid=x_valid.reshape(n_valid,28,28,1)

x_valid=x_valid/255.0

yPred=cnn_net.predict_classes(x_valid,batch_size=32,verbose=1)

np.savetxt('Digit3.csv',np.c_[range(1,len(yPred)+1),yPred],delimiter=',',header='ImageId,Label',comments='',fmt='%d')

28000/28000 [==============================] - 24s 841us/step
